In [1]:
%pdb on

Automatic pdb calling has been turned ON


In [2]:
import numpy as np
import tensorflow as tf
import sys
import os
if os.path.abspath('../') not in sys.path:
    sys.path.append(os.path.abspath('../'))
import automatic_speech_recognition as asr
import time

# Train/Eval Part

In [3]:
dev_dataset = asr.dataset.Audio.from_csv('./data/libri_dev/data_5s.csv', batch_size=1, use_filesizes=True, relative_paths=False)
alphabet = asr.text.Alphabet(lang='en')
features_extractor = asr.features.MFCC(
    features_num=26,
    is_standardization=False,
    winlen=0.032,
    winstep=0.02,
)

In [4]:
# model = asr.model.get_deepspeech(
#     input_dim=26,
#     output_dim=29,
#     units=1024,
#     random_state=24,
#     dropouts=(0,0.0,0.0,0.0,0),
#     context=9,
#     # rnn_units=800,
#     # is_mixed_precision=False,
# )
model = asr.model.load_mozilla_deepspeech('./data/mozilla_deepspeech.pb')
optimizer = tf.optimizers.Adam(
    lr=1e-3,
    beta_1=0.9,
    beta_2=0.999
)
decoder = asr.decoder.GreedyDecoder()
pipeline = asr.pipeline.CTCPipeline(
    alphabet, features_extractor, model, optimizer, decoder
)
callbacks = []
# pipeline.fit(dataset, dev_dataset, epochs=1000, callbacks=callbacks)
# pipeline.save('./checkpoint')

load graph
Instructions for updating:
Use tf.gfile.GFile.
['IteratorV2', 'IteratorGetNext', 'Shape', 'strided_slice/stack', 'strided_slice/stack_1', 'strided_slice/stack_2', 'strided_slice', 'Const', 'conv1d/ExpandDims/dim', 'conv1d/ExpandDims', 'conv1d/ExpandDims_1/dim', 'conv1d/ExpandDims_1', 'conv1d', 'conv1d/Squeeze', 'Reshape/shape/1', 'Reshape/shape/2', 'Reshape/shape/3', 'Reshape/shape', 'Reshape', 'transpose/perm', 'transpose', 'Reshape_1/shape', 'Reshape_1', 'layer_1/bias', 'layer_1/bias/read', 'layer_1/weights', 'layer_1/weights/read', 'MatMul', 'BiasAdd', 'Relu', 'Minimum/y', 'Minimum', 'StringFormat', 'PrintV2', 'Identity', 'layer_2/bias', 'layer_2/bias/read', 'layer_2/weights', 'layer_2/weights/read', 'MatMul_1', 'BiasAdd_1', 'Relu_1', 'Minimum_1/y', 'Minimum_1', 'layer_3/bias', 'layer_3/bias/read', 'layer_3/weights', 'layer_3/weights/read', 'MatMul_2', 'BiasAdd_2', 'Relu_2', 'Minimum_2/y', 'Minimum_2', 'Reshape_2/shape', 'Reshape_2', 'cudnn_lstm/rnn/multi_rnn_cell/cell_0/

In [5]:
start = time.time()
wer, cer = asr.evaluate.calculate_error_rates(pipeline, dev_dataset, print_pred=True)
elapsed_time = time.time() - start
print(f'WER: {wer}   CER: {cer}')
print(f'Elapsed time {elapsed_time}')

['shortly after passing one of these chapels we came suddenly upon a village which started up out of the mist and i was alarmed lest i should be made an object of curiosity or dislike']
['my guids however were well known and the natural politeness of the people prevented them from putting me to any inconvenience but they could not help eyeing me nor ei them']
['the streets were narrow an unpaved bat very fairly clean']
['the vine gr outside many of the houses and there were some with signboards on which was painted abottle an the glass that made me feel much at home']
['even on this lege of human society there was a stunted girwh of shoplets which had taken invegetated somehow though as in an air marcantile of the bleakest']
WER: 0.13806451612903228   CER: 0.035877050349450865
Elapsed time 17.380148887634277


In [6]:
model.summary()

Model: "DeepSpeech"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X (InputLayer)                  [(None, None, 26)]   0                                            
__________________________________________________________________________________________________
tf_op_layer_ExpandDims (TensorF [(None, 1, None, 26) 0           X[0][0]                          
__________________________________________________________________________________________________
tf_op_layer_Shape (TensorFlowOp [(3,)]               0           X[0][0]                          
__________________________________________________________________________________________________
tf_op_layer_Conv2D (TensorFlowO [(None, 1, None, 494 0           tf_op_layer_ExpandDims[0][0]     
_________________________________________________________________________________________

# Export Part

In [9]:
model = asr.model.load_mozilla_deepspeech('./data/mozilla_deepspeech.pb', tflite_version=True)

exporter = asr.utils.export.KerasTfLiteExporter(model, skip_on_load_fail=True)
exporter.experimental_new_converter = True
exporter.allow_custom_ops = True
exporter.export('./ds.tflite')


load graph
['IteratorV2', 'IteratorGetNext', 'Shape', 'strided_slice/stack', 'strided_slice/stack_1', 'strided_slice/stack_2', 'strided_slice', 'Const', 'conv1d/ExpandDims/dim', 'conv1d/ExpandDims', 'conv1d/ExpandDims_1/dim', 'conv1d/ExpandDims_1', 'conv1d', 'conv1d/Squeeze', 'Reshape/shape/1', 'Reshape/shape/2', 'Reshape/shape/3', 'Reshape/shape', 'Reshape', 'transpose/perm', 'transpose', 'Reshape_1/shape', 'Reshape_1', 'layer_1/bias', 'layer_1/bias/read', 'layer_1/weights', 'layer_1/weights/read', 'MatMul', 'BiasAdd', 'Relu', 'Minimum/y', 'Minimum', 'StringFormat', 'PrintV2', 'Identity', 'layer_2/bias', 'layer_2/bias/read', 'layer_2/weights', 'layer_2/weights/read', 'MatMul_1', 'BiasAdd_1', 'Relu_1', 'Minimum_1/y', 'Minimum_1', 'layer_3/bias', 'layer_3/bias/read', 'layer_3/weights', 'layer_3/weights/read', 'MatMul_2', 'BiasAdd_2', 'Relu_2', 'Minimum_2/y', 'Minimum_2', 'Reshape_2/shape', 'Reshape_2', 'cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/kernel', 'cudnn_lstm

In [10]:
model.summary()

Model: "DeepSpeech"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X (InputLayer)                  [(None, 3, 26)]      0                                            
__________________________________________________________________________________________________
tf_op_layer_ExpandDims_3 (Tenso [(None, 1, 3, 26)]   0           X[0][0]                          
__________________________________________________________________________________________________
tf_op_layer_Shape_3 (TensorFlow [(3,)]               0           X[0][0]                          
__________________________________________________________________________________________________
tf_op_layer_Conv2D_3 (TensorFlo [(None, 1, 3, 494)]  0           tf_op_layer_ExpandDims_3[0][0]   
_________________________________________________________________________________________

In [12]:
l = model.layers[9]

In [15]:
model.save('ds.h5', save_format='h5')

In [16]:
ls

DeepSpeechExperiments.ipynb  deepspeech_speedup.ipynb     ds.tflite                    models/
data/                        ds.h5                        model.png                    tflite_models/
